In [1]:
#list of the stocks

Initial_List = ["AAPl", "FE", "TSLA", "PEP", "MNOV", "SHOP", "IRBT", "SIX", "NNVC", "HBSC", "NOC", "RTN", "PBR", "BBD", "IBM", "SAP", "ACN", "TXN", "NKE", "DIS", "JPM", "AME", "MSG", "SBUX", "CNNE", "GIS", "PG", "KO", "AMRS", "HEI", "MSFT", "AMD", "AMZN", "FB", "NFLX", "ORCL", "SPCB", "ELY", "CVX", "RHNO", "CECO"]

In [2]:
#Getting the historical Data by symbol
def GetHistoricalData(symbol):
  try:
    start = datetime.datetime(2016, 1, 1)
    end = datetime.datetime.today().date()
    result = get_historical_data(symbol, start, end, token=IEX_token)
    WriteToDataLake(result, symbol +"/Hist/")
  except:
    pass


In [3]:
#Writing to DataLake

def WriteToDataLake(pdf, folderpath):
#   df = pandas_to_spark(pdf)
  df = spark.createDataFrame(pdf)
  df.repartition(1).write.mode("append").format('csv').options(header='true', delimiter=',', inferSchema='true').save("abfss://iex@hhaggandl2.dfs.core.windows.net/MMAI_Finance2/"+folderpath)

In [4]:
import pandas as pd
import quandl
import datetime

from iexfinance.stocks import Stock
from iexfinance.altdata import get_social_sentiment
from iexfinance.stocks import get_historical_data
from iexfinance.refdata import get_symbols
from iexfinance.stocks import get_historical_intraday

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.functions import explode
from pyspark.sql.types import *

from pyspark.sql.types import *
from pyspark.sql.functions import from_json, to_json
from pyspark.sql.types import DateType, StringType, StructType
import json

In [5]:
#Initializaing the main services

#region IEX Configuration

IEX_URL = "abfss://iex@hhaggandl2.dfs.core.windows.net/MMAI/"

#region Quandl password
quandl.ApiConfig.api_key = "7tPpbtseQHTGtt4RjPci"
#endregion

#region IEX password
IEX_token = "sk_37e05de0f447447bba587440923da37a"
#endregion

#endregion

#region Datalake Configuration
spark.conf.set("fs.azure.account.key.hhaggandl2.dfs.core.windows.net", "EjU3j412niQaDfXUG5XDVaybkszqiPCcO9+x6fXTRNUnhpgV5Dsu6Fg33uItZ/bLXXHIYfQrDBmC7Bu/ADQJ9A==")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://iex@hhaggandl2.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

#endregion

#region Symbols

def GetSymbols():
    return get_symbols(output_format='pandas', token=IEX_token)

# #endregion

All_Symbols = GetSymbols()


In [6]:
import io
import pandas
import requests
import datetime

base = "https://www.quandl.com/api/v3/datasets/WIKI/"
api = "7tPpbtseQHTGtt4RjPci"

def fun(stock):
  resp = requests.get(base+stock+'/data.csv?api_key='+ api)
  s = resp.content
  df = pd.read_csv(io.StringIO(s.decode('utf-8')))
  WriteToDataLake(df, stock)
  

In [7]:
for stock_symbol in Initial_List:
  fun(stock_symbol)

In [8]:
for stock_symbol in All_Symbols.symbol:
  fun(stock_symbol)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-4179646198054075> in <module> () 
 1 for stock_symbol in All_Symbols . symbol : 
 ----> 2 fun ( stock_symbol ) 

 <command-873924226432057> in fun (stock) 
 11 s = resp . content 
 12 df = pd . read_csv ( io . StringIO ( s . decode ( 'utf-8' ) ) ) 
 ---> 13 WriteToDataLake ( df , stock ) 
 14 

 <command-3841365661557872> in WriteToDataLake (pdf, folderpath) 
 3 def WriteToDataLake ( pdf , folderpath ) : 
 4 # df = pandas_to_spark(pdf) 
 ----> 5 df = spark . createDataFrame ( pdf ) 
 6 df . repartition ( 1 ) . write . mode ( "append" ) . format ( 'csv' ) . options ( header = 'true' , delimiter = ',' , inferSchema = 'true' ) . save ( "abfss://iex@hhaggandl2.dfs.core.windows.net/MMAI_Finance2/" + folderpath ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 815 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 816 else : 
 --> 817 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 818 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 819 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 440 write temp files . 
 441 """
 --> 442 data , schema = self . _wrap_data_schema ( data , schema ) 
 443 return self . _sc . parallelize ( data ) , schema 
 444 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 419 
 420 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 421 struct = self . _inferSchemaFromList ( data , names = schema ) 
 422 converter = _create_converter ( struct ) 
 423 data = map ( converter , data ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchemaFromList (self, data, names) 
 350 """
 351 if not data : 
 --> 352 raise ValueError ( "can not infer schema from empty dataset" ) 
 353 first = data [ 0 ] 
 354 if type ( first ) is dict : 

 ValueError : can not infer schema from empty dataset